# Looking at the historical US market returns

## Imports

In [48]:
import pandas as pd
import matplotlib.pyplot as plt

## Data

### Shiller's historical S&P 500 data

In [49]:
df = pd.read_excel('../data/shiller_sp500.xls', sheet_name='Data', header=[1,2,3,4,5,6,7])

#### Trim down and join column names

There's some weird formatting due to a preamble above the first several columns.

In [59]:
columns = df.columns.to_flat_index().tolist()

for index, words in enumerate(columns):
    # Remove all the preamble text
    if not words[0].startswith('Cyclically'):
        words = words[-4:]
    # Remove the empty column entries
    while words[0].startswith('Unnamed'):
        words = words[1:]
    columns[index] = words
    print(columns[index])

('Date',)
('S&P', 'Comp.', 'P')
('S&P', 'Dividend', 'D')
('S&P', 'Earnings', 'E')
('  Consumer', 'Price', 'Index', 'CPI')
('  Consumer', 'Price', 'Date  ', 'Fraction')
('  Consumer', 'Long', 'Interest', 'Rate GS10')
('  Consumer', 'Long', 'Real', 'Price')
('  Consumer', 'Long', 'Real', 'Dividend')
('Real', 'Total', 'Return', 'Price')
('Real', 'Total', 'Real', 'Earnings')
('Real', 'TR', 'Scaled', 'Earnings')
('Cyclically', 'Adjusted', 'Price', 'Earnings', 'Ratio', 'P/E10 or', 'CAPE')
('Cyclically', 'Adjusted', 'Price', 'Earnings', 'Ratio', 'P/E10 or', 'CAPE.1')
('Cyclically ', 'Adjusted', 'Total Return Price', 'Earnings', 'Ratio', 'TR P/E10 or', 'TR CAPE')
('Cyclically ', 'Adjusted', 'Total Return Price', 'Earnings', 'Ratio', 'TR P/E10 or', 'TR CAPE.1')
('Cyclically ', 'Adjusted', 'Total Return Price', 'Earnings', 'Excess', 'CAPE', 'Yield')
('Cyclically ', 'Adjusted', 'Total Return Price', 'Monthly', 'Total', 'Bond', 'Returns')
('Cyclically ', 'Adjusted', 'Total Return Price', 'Real', '